In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")

llm.invoke("What is the capital of France?")


AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-03-25T05:45:54.9867031Z', 'done': True, 'done_reason': 'stop', 'total_duration': 410206200, 'load_duration': 17231400, 'prompt_eval_count': 32, 'prompt_eval_duration': 50902100, 'eval_count': 8, 'eval_duration': 341555800, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-20b375d6-2369-4402-a62d-1d287269fca6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40})

답변 형식을 컨트롤하는 방법은 두가지
- StrOutputParser 사용
- Pydantic 모델 사용

### 1. StrOutputParser 사용 

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"]
)

prompt = prompt_template.invoke({"country": "France"})

print(prompt)

ai_message = llm.invoke(prompt_template.invoke({"country": "France"}))

output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"})))



text='What is the capital of France? Return the name of the city only'


In [23]:
ai_message

AIMessage(content='Paris.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-03-25T06:02:00.0690013Z', 'done': True, 'done_reason': 'stop', 'total_duration': 351355300, 'load_duration': 17265300, 'prompt_eval_count': 39, 'prompt_eval_duration': 236636300, 'eval_count': 3, 'eval_duration': 96936400, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-5301fd89-461d-4be1-b98e-c517776649bc-0', usage_metadata={'input_tokens': 39, 'output_tokens': 3, 'total_tokens': 42})

In [24]:
type(ai_message.content)
# json이 아닌 str인점 주의

str

In [25]:
answer

'Paris'

### 구조화된 출력 사용

Pydantic 모델을 사용하여 LLM의 출력을 구조화된 형식으로 받을 수 있다.

In [15]:
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of the country")
    population: int = Field(description="The population of the country")
    language: str = Field(description="The language of the country")
    currency: str = Field(description="The currency of the country")


structued_llm = llm.with_structured_output(CountryDetail)

In [18]:
country_detail_prompt = PromptTemplate(
    template="""Give following information about {country}:
    - Capital
    - Population
    - Language
    - Currency

    return it in JSON format. and return the JSON dictionry only 
    """,
    input_variables=["country"],
)

country_detail_prompt.invoke({"country": "France"})

json_ai_message = structued_llm.invoke(country_detail_prompt.invoke({"country": "France"}))


In [19]:
(json_ai_message)

CountryDetail(capital='Paris', population=67, language='French', currency='$')

In [21]:
json_ai_message.model_dump()["capital"]
# # Pydantic 모델의 특정 필드로 접근이 가능하다.

'Paris'